In [1]:
from OTXv2 import OTXv2
from OTXv2 import IndicatorTypes
from stix2 import Indicator, ObservedData, ExternalReference
otx = OTXv2("7ef390234ffa11dd340269f1ae184cf9fe3191bd2ba16b23148fe33c4ebe0c91")

In [3]:
import requests

def search_domain(domain, section):
    endpoint = f"https://otx.alienvault.com/api/v1/indicators/domain/{domain}/{section}"
    try:
        response = requests.get(endpoint)
        response.raise_for_status()  # raise an exception for non-200 status codes
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None


In [4]:
import json
from stix2 import Indicator, parse

# Mapping of section names to STIX object types
SECTION_TYPE_MAP = {
    'general': 'indicator',
    'geo': 'location',
    'malware': 'malware',
    'url_list': 'url',
    'passive_dns': 'domain-name',
    'whois': 'identity',
    'http_scans': 'network-traffic',
}

def json_to_stix(domain, section):
    # Load JSON data from API response
    json_data = search_domain(domain, section)
    
    # Determine the corresponding STIX object type
    stix_type = SECTION_TYPE_MAP.get(section, 'indicator')
    
    # Create STIX 2.1 object from JSON data
    if stix_type == 'indicator':
        stix_object = Indicator.from_dict(json_data)
    else:
        stix_object = parse(json.dumps(json_data), stix_type)
    
    return stix_object


In [5]:
from stix2 import parse
import json

# Search for domain in API and get JSON data
data = search_domain("example.com", "geo")
data['type'] = 'location'

# Convert the dictionary to JSON string
json_data = json.dumps(data)

# Create STIX 2.1 indicator from JSON string using parse()
indicator = parse(json_data, allow_custom=True, version="2.1")

# Print the indicator in JSON format
print(indicator.serialize())

{"type": "location", "spec_version": "2.1", "id": "location--0384cf32-0aca-4ad7-9bcc-27879e7e2904", "created": "2023-04-13T02:15:57.969606Z", "modified": "2023-04-13T02:15:57.969606Z", "latitude": 42.1591, "longitude": -70.8163, "region": "MA", "city": "Norwell", "postal_code": "02061", "accuracy_radius": 1000, "area_code": 0, "asn": "AS15133 verizon", "charset": 0, "city_data": true, "continent_code": "NA", "country_code": "US", "country_code2": "US", "country_code3": "USA", "country_name": "United States of America", "dma_code": 506, "flag_title": "United States of America", "flag_url": "/assets/images/flags/us.png", "subdivision": "MA"}


In [ ]:
import json
from stix2 import Indicator, parse

# Mapping of section names to STIX object types
SECTION_TYPE_MAP = {
    'general': 'indicator',
    'geo': 'location',
    'malware': 'malware',
    'url_list': 'url',
    'passive_dns': 'domain-name',
    'whois': 'identity',
    'http_scans': 'network-traffic',
}

def json_to_stix(domain, section):
    # Load JSON data from API response
    json_data = search_domain(domain, section)
    
    # Determine the corresponding STIX object type
    stix_type = SECTION_TYPE_MAP.get(section, 'indicator')
    
    # Create STIX 2.1 object from JSON data
    if stix_type == 'indicator':
        stix_object = Indicator.from_dict(json_data)
    else:
        stix_object = parse(json.dumps(json_data), stix_type)
    
    return stix_object


In [ ]:
# Example usage
domain = "ualr.edu"
section = "http_scans"
"""
sections:
general: General information about the domain, including any pulses, and a list of the other sections currently available for this domain.
geo: A more verbose listing of geographic data (Country code, coordinates, etc.)
malware: Malware samples analyzed by AlienVault Labs which have been observed connecting to this domain.
url_list: URLs analyzed by AlienVault Labs on this domain.
passive_dns: Passive dns records observed by AlienVault Labs pointing to this domain.
whois: Whois records for the domain.
http_scans: Meta data for http(s) connections to the domain.
"""
data = search_domain(domain, section)
if data:
    print(data)
else:
    print(f"Error: Failed to retrieve {section} data for {domain}")